---
Ce programme nous permet de traiter les données Events de Stats Bomb pour en extraire :
1) Les centres ainsi qu'un certain nombre d'events les suivant (ce ne nombre est défini à l'aide de la variable nb_events_suivant)
2) Les positions de ces centres et de leur zone de réception
3) Pour ces events suivant les centres, on repère les tirs et les buts inscrits par l'équipe qui a centré
4) Les positions de ces tirs ainsi que des endroits ou ils se sont terminés
5) Des informations liées à ces events telles que la date du match, la journée, les équipes à domicile et à l'extérieur, la saison, la compétition ainsi que le "match_id" Stats Bomb

Grâce à ces données, nous pourrons afficher sur un terrain, via l'application Web Streamlit :
- Des cartes de chaleurs "heatmaps" des zones de centre et de réception de centre
- Des flèches représentant la trajectoire des tirs survenus dans les "nb_events_suivant" events suivant les centres
- Les zones ou se terminent les tirs dans la cage du gardien
---

## 1) Importation des bibliothèques

In [1]:
import pandas as pd
import sqlite3
import sys

---

## 2) Connexion à la BDD

In [2]:
connect = sqlite3.connect("../../Importation_donnees/raw-database.db")
cursor = connect.cursor()

---

## 3) Importation des données events

In [3]:
colonnes = ["x_loc", "y_loc", "event_id", "match_id_SB", "type", "team_id_SB", "minute", "shot_outcome", "pass_type", "period",
            "pass_cross", "index_event", "x_shot", "y_shot", "z_shot", "x_pass", "y_pass", "pass_body_part", "player_id_SB"]
colonnes = ", ".join(colonnes)
req = cursor.execute(f"SELECT {colonnes} FROM events")
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [4]:
# Aperçu des données
events.head(2)

,x_loc,y_loc,event_id,match_id_SB,type,team_id_SB,minute,shot_outcome,pass_type,period,pass_cross,index_event,x_shot,y_shot,z_shot,x_pass,y_pass,pass_body_part,player_id_SB
0,NaN,NaN,02591dfb-4789-442f-ae99-53ac3f3aca81,3894037,Starting XI,147,0,None,None,1,NaN,1,NaN,NaN,NaN,NaN,NaN,None,NaN
1,NaN,NaN,a5620f61-b542-46c8-b0b4-8e2daf267e84,3894037,Starting XI,156,0,None,None,1,NaN,2,NaN,NaN,NaN,NaN,NaN,None,NaN


Avant de se lancer dans la suite du traitement, nous allons modifier l'index "index_event", qui n'identifie plus les events de manière unique.  
En effet, de base, cet index permet d'identifier de manière unique l'ensemble des events d'un match, de manière chronologique : l'event d'un match ayant pour "index_event" n correspond au n-ème event référencé de ce match.  
Cependant, étant donné que la table "events" contient les events de plusieurs matchs, cet index "index_event" n'est plus unique.  
Nous pouvons le constater grâce à la cellulle suivante :

In [ ]:
events.index_event.is_unique

False

Nous allons donc re-créer cette variable "index_event", afin qu'elle identifie de nouveau les events de manière unique, et par ordre chronologique.  
L'ordre chronologique est assuré par le fait que dans le programme d'importation, nous avons trié les events par "match_id_SB" et en fonction de leur "index_event"

In [6]:
# Réindexation des events
events["index_event"] = range(len(events))

---
Maintenant que nous avons importé les données Events, nous allons extraire les events correspondant à des centres.  
Nb : nous avons préféré ne pas analyser les centres qui sont des corners

## 4) Récupération des events liés à des centres

In [7]:
# Création d'un dataframe ne comprenant que les events liés à des centres (excepté les corners)
centres = events[(events.pass_cross == 1) & (events.pass_type != "Corner")]

---
Dans la prochaine section, nous allons récupérer les events suivant ces centres

## 5) Extraction des events suivant les centres

In [8]:
# Importation de la variable "nb_events_suivant", définie dans le programme "variable.py" qui est stocké dans le dossier "Application_streamlit"

# Ajout du chemin vers le dossier "Application_streamlit"
sys.path.append('../../../Application_streamlit')

# On récupère la variable dans le programme "variable.py"
from variables import nb_events_suivant

print(nb_events_suivant)

5


Nous souhaitons donc récupérer les 5 events suivant chaque centre.  
Pour ce faire, nous allons donc utiliser la colonne "index_event".  
Exemple : si nous souhaitons récupérer les 5 events suivant un centre ayant pour "index_event" 3, alors nous devrons extraire les events ayant pour "index_event" 4, 5, 6, 7 et 8.  

Cependant, il faut faire attention, car plusieurs cas peuvent poser problème.  
Par exemple, s'il y a un autre centre dans les 5 events, ou bien si le centre correspond au dernier event d'une mi-temps, et donc les 5 events suivant ce centre se sont déroulés lors d'une autre période/match et il est donc incohérent des les analyser.

### a) Création du dataframe

Dans un premier temps, nous allons créer un "modèle" du dataframe qui contiendra les events liés aux centres et les 5 events les suivant (même s'ils se sont déroulés lors d'une autre mi-temps ou lors d'un autre match).  
Ce dataframe aura donc une taille de 6 fois le nombre de centres.  
En effet, il contiendra une ligne par centre, et une ligne pour chaque event se déroulant dans les 5 events suivant un centre.  
Dans le cas ou un autre centre se déroulerait dans les 5 events suivant un centre, certains events seront allors dupliqués.


Ce dataframe contiendra 5 colonnes/informations :
- L'équipe "team_centre" et le joueur "Centreur" ayant effectué le centre, qui seront les même pour le centre et les events suivant ce centre. L'information "team_centre" nous permettra de savoir si l'équipe liée à un des events suivant le centre est la même que l'équipe qui a effectué le centre associé à cet event
- Le "centre_id", qui sera le même pour le centre et les events suivant ce centre. Il permettra d'associé le centre et les events le suivant
- Les "match_id_SB" et les "period" liés au centre, qui seront eux aussi attribués aux events suivant. Il est donc possible que le "match_id_SB" ou la "period" attribué à un event ne soit pas la même que le réel "match_id_SB"/"period" de cet event. Par exemple, dans le cas ou le centre est le dernier event de la 1ère mi-temps et donc la "period" des 5 events le suivant ne correspondent pas
- La colonne "index_event", qui contiendra les "index_event" de l'ensemble des centres et des events les suivant. Elle permettra d'effectuer la jointure avec les events du dataframe "events", et donc d'extraire les events suivant les centres

In [9]:
centres = pd.DataFrame({"team_centre" : [i for i in centres.team_id_SB for _ in range (nb_events_suivant + 1)],
                    "Centreur" : [i for i in centres.player_id_SB for _ in range (nb_events_suivant + 1)],
                    "centre_id" : [i for i in range (1, len(centres) + 1) for _ in range (nb_events_suivant + 1)],
                    "match_id_SB" : [i for i in centres.match_id_SB for _ in range (nb_events_suivant + 1)],
                    "period" : [i for i in centres.period for _ in range (nb_events_suivant + 1)],
                    "index_event" : [j + i for j in centres.index_event for i in range (nb_events_suivant + 1)]})

La méthode "for _ in range (nb_events_suivant + 1)" permet de répéter (nb_events_suivant + 1) fois la valeur itérée par i.  
Par exemple, si nb_events_suivant vaut 5 et i prend les valeurs 1, 2, 3, etc., alors cela donnera :
[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, etc.].  
C'est cette méthode qui nous permet d'attribuer aux events suivant les centres les informations liées au centre, telles que l'équipe qui l'a effectué, la période ou il a été effectué etc.

In [10]:
# Aperçu des données
centres.head(10)

,team_centre,Centreur,centre_id,match_id_SB,period,index_event
0,147,26532.0,1,3894037,1,153
1,147,26532.0,1,3894037,1,154
2,147,26532.0,1,3894037,1,155
3,147,26532.0,1,3894037,1,156
4,147,26532.0,1,3894037,1,157
5,147,26532.0,1,3894037,1,158
6,147,316514.0,2,3894037,1,259
7,147,316514.0,2,3894037,1,260
8,147,316514.0,2,3894037,1,261
9,147,316514.0,2,3894037,1,262


### b) Jointure avec les events du dataframe "events"

Nous pouvons donc effectuer la jointure de ce dataframe avec le dataframe "events", par rapport aux "index_event", "match_id_SB" et "period".  
Ces colonnes utilisées pour la jointure nous permettent de supprimer les cas ou un centre se serait déroulé dans les "nb_events_suivant" derniers events d'une période, et donc de supprimer les events les suivant qui se sont déroulés dans une autre mi-temps.

In [11]:
events_centres = pd.merge(centres, events, on = ["index_event", "match_id_SB", "period"])

In [12]:
# Aperçu des données
events_centres.head(10)

,team_centre,Centreur,centre_id,match_id_SB,period,index_event,x_loc,y_loc,event_id,type,...,shot_outcome,pass_type,pass_cross,x_shot,y_shot,z_shot,x_pass,y_pass,pass_body_part,player_id_SB
0,147,26532.0,1,3894037,1,153,101.8,3.0,872df8f3-a750-4b61-8890-61a35e96b4a4,Pass,...,None,None,1.0,NaN,NaN,NaN,116.4,37.6,Left Foot,26532.0
1,147,26532.0,1,3894037,1,154,108.7,31.8,0f842e18-1363-4af3-8faa-6c9cce55b118,Ball Receipt*,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,63413.0
2,147,26532.0,1,3894037,1,155,3.7,42.5,a8a08a44-9486-4ce7-ab31-d6bd6f5cff23,Ball Recovery,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,24987.0
3,147,26532.0,1,3894037,1,156,3.7,42.5,3c668f97-7d4c-46e4-a4cd-3637d134b1f1,Carry,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,24987.0
4,147,26532.0,1,3894037,1,157,15.4,45.6,a42ba11b-280b-405c-9674-6ccebf0a67ad,Pass,...,None,None,NaN,NaN,NaN,NaN,31.4,12.3,Keeper Arm,24987.0
5,147,26532.0,1,3894037,1,158,31.4,12.3,b3fa9602-6342-4e5a-b3e4-9312064c077c,Ball Receipt*,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,137710.0
6,147,316514.0,2,3894037,1,259,106.2,61.8,102a8d71-103b-4a16-a16a-09315649df86,Pass,...,None,None,1.0,NaN,NaN,NaN,108.2,46.4,Right Foot,316514.0
7,147,316514.0,2,3894037,1,260,110.1,40.5,a193f7dd-36e8-4fe0-98bd-2c546bf70abe,Ball Receipt*,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,63413.0
8,147,316514.0,2,3894037,1,261,11.9,33.7,44d60e1c-f666-4735-a801-8b015a168d29,Clearance,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,22098.0
9,147,316514.0,2,3894037,1,262,120.0,80.0,29f8a9b6-c323-4c4e-a5c8-5d918ab64c3f,Pass,...,None,Corner,NaN,NaN,NaN,NaN,108.9,38.0,Right Foot,7252.0


### c) Cas ou il y a un autre centre dans les "nb_events_suivant" events suivant un centre

Nous allons maintenant traiter les cas ou il y a un centre dans les events suivant un centre.  
Dans ces cas, nous allons uniquement garder le 2ème centre et les events suivant ce dernier.  
Dans le cas ou il y aurait 3 centres qui se suivraient, alors on garderait le 3ème centre et les events suivant ce dernier, et de même pour + de centres se suivant...  

Exemple d'un cas qui pourrait poser problème : on a un centre qui a pour "index_event" 5 et "centre_id" 2, et 3 events après ce centre, il y a eu un autre centre (qui a donc pour "index_event" 8 et "centre_id" 3).  
Les valeurs actuelles de la colonne "index_event" du dataframe "events_centres" sont donc :  
[5, 6, 7, 8, 9, 10, 8, 9, 10, 11, 12, 13].  
Tandis que les valeurs de la colonne "centre_id" sont :  
[2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3].  
Donc souhaitons donc supprimer les lignes ayant pour "centre_id" 2

Pour repérer ces cas, il suffit donc de repérer les "index_event" dupliqués.  
Pour garder le centre et les events le suivant, nous allons utiliser la commandes "duplicated" de Pandas.  

La fonction "duplicated" permet d'indiqué si une valeur d'une ou plusieurs colonnes à la fois est dupliquée. Elle n'indique pas comme "dupliquée" la première occurence d'une valeur dupliquée.  
Par exemple, s'il y'a 2 lignes qui contiennent la valeur "ok", alors seulement la deuxième ligne sera marquée comme dupliquée.  
Elle va nous permettre de repérer les "centre_id" dupliqués.

Nous regardons les valeurs dupliquées pour les colonnes "team_id_SB" et "index_event".  
En effet, si c'est une autre équipe qui a effectué le centre se déroulant dans les "nb_events_suivant" events suivant le 1er centre, alors c'est incohérent de ne pas garder le 1er centre

In [13]:
# Récupération des "centre_id" dupliqués
centre_id_dupl = events_centres.loc[events_centres[["team_id_SB", "index_event"]].duplicated(), "centre_id"].unique() - 1
# Nous ajoutons la commande "unique()" car nous souhaitons seulement connaitre la liste des "centre_id" dupliqués, de manière unique.
# Nous ajoutons "- 1" car nous souhaitons garder les centres suivant d'autres centres, en d'autres termes supprimer les centres pour lesquels il
# y a un autre centre dans les events suivant ces centres.

Cependant, nous n'allons pas supprimer maintenant ces centres.  
En effet, dans le cas ou l'équipe ayant effectué un de ces centres marquerait lors d'un event se déroulant entre ce centre et le centre effectué dans les "nb_events_suivant" events suivant ce centre, alors on souhaiterait garder les informations liés à ce centre.  
Explication de ce cas avec un exemple :  
Prenons nb_events_suivant = 10.  
Si une équipe effectue un centre (qu'on nomme centre 1), qu'elle effectue un autre centre (centre 2) correspondant au 9ème event suivant "centre 1", mais qu'elle marque lors du deuxième event suivant "centre 1", alors nous souhaitons garder "centre 1" ainsi que les events le suivant, même si cette équipe a effectué un autre centre dans les "nb_events_suivant" events suivant "centre 1".  
Nous supprimerons donc ces centres après avoir detécté les centres qui ont amenés un but dans les "nb_events_suivant" events suivant ces centres

---
Nous allons maintenant passer au traitement des tirs dans les "nb_events_suivant" events suivant les centres.  
Nous nous intéresserons seulement aux tirs effectués par l'équipe qui a centré.  
Nous souhaitons en extraire la position de départ des tirs (x_loc et y_loc), leur position d'arrivée (x_shot, y_shot et z_shot) et le joueur qui a tiré.

## 6) Traitement des tirs dans les events suivant les centres

Avant cela, nous allons supprimer les lignes ayant des "index_event" dupliqués, pour + de clarté

In [14]:
events_centres.drop_duplicates("index_event", keep = "last", inplace = True)

In [15]:
# Nous créons une nouvelle colonne "Tireur" qui contiendra, pour chaque event correspondant à un tir, le joueur ayant effectué le tir
events_centres["Tireur"] = ""

In [16]:
# Récupération des lignes correspondant à des tirs
# La condition "events_centres.team_centre == events_centres.team_id_SB" nous assure que nous gardons seulement les tirs effectués par les équipes
# qui ont effectué le centre associé à l'event lié au tir
filtre_shot = (events_centres.type == "Shot") & (events_centres.team_centre == events_centres.team_id_SB)

# Ajout de l'information sur le tireur à la colonne "Tireur"
events_centres.loc[filtre_shot, "Tireur"] = events_centres.loc[filtre_shot, "player_id_SB"]

In [17]:
# Aperçu des données
ex_tir = events_centres.index[filtre_shot][1]
events_centres.loc[ex_tir - 2 : ex_tir + 2]

,team_centre,Centreur,centre_id,match_id_SB,period,index_event,x_loc,y_loc,event_id,type,...,pass_type,pass_cross,x_shot,y_shot,z_shot,x_pass,y_pass,pass_body_part,player_id_SB,Tireur
30,147,63413.0,6,3894037,1,584,117.4,65.3,be451a2a-030f-424d-bfc8-1bc92e6c6061,Pass,...,None,1.0,NaN,NaN,NaN,107.5,43.4,Left Foot,63413.0,
31,147,63413.0,6,3894037,1,585,107.5,43.4,e3f35afe-fceb-428d-a788-7b15bc53bf1d,Ball Receipt*,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,80484.0,
32,147,63413.0,6,3894037,1,586,107.5,43.4,dfee00d8-8882-4349-8766-196daeecc7b0,Shot,...,None,NaN,107.8,35.2,NaN,NaN,NaN,None,80484.0,80484.0
33,147,63413.0,6,3894037,1,587,1.0,39.0,0ddd0b6d-fcfb-4531-81b9-2d32b44e236b,Goal Keeper,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,24987.0,
34,147,63413.0,6,3894037,1,588,108.2,28.5,91388ad5-c2c6-4f10-9edc-f3529228e9e4,Ball Recovery,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,46258.0,


---
Après avoir analysé les tirs effectués dans les "nb_events_suivant" events suivant des centres, nous allons traiter les centres ayant amené à des buts dans les "nb_events_suivant" events les suivant

## 7) Traitement des centres ayant amené à des buts

In [18]:
# Dans un premier temps, nous créons une colonne "But" qui contient la valeur "Non" pour chaque ligne du dataframe
# Nous modifierons ensuite cette colonne pour les lignes correspondant à des centres ayant amené à un but ou à des events correspondant à des buts
events_centres["But"] = "Non"

In [19]:
# Création d'un filtre pour récupérer les lignes/events liées à des buts
# Pour pouvoir extraire les events liés à des CSC, nous ajoutons la condition "events_centres.type == "Own Goal For""
filtre_but = ((events_centres.shot_outcome == "Goal") | (events_centres.type == "Own Goal For")) & (events_centres.team_centre == events_centres.team_id_SB)

# Ajout de l'information du but aux lignes liées à des buts
events_centres.loc[filtre_but, "But"] = "Oui"

In [20]:
# Nous souhaitons aussi indiquer les centres ayant amené à des buts, c'est ce que nous réalisons avec la ligne suivante
# Pour cela, nous allons récupérer les "centre_id" des lignes correspondant à des buts
centre_id_but = events_centres.loc[filtre_but, "centre_id"]

# Modification de la colonne "But" pour les centres ayant amené à un but
events_centres.loc[(events_centres.centre_id.isin(centre_id_but)) & (events_centres.pass_cross == 1), "But"] = "Oui"

A présent, nous connaissons grâce à la colonne "But", l'ensemble des centres ayant amené à des buts mais aussi les events liés à des buts.

Nous pouvons donc supprimer les centres (et les events qui y sont liés) pour lesquels un autre centre est survenu dans les "nb_events_suivant" events les suivant.  
Nous avions détecté ces centres dans la partie c) de la section 5).

In [21]:
# Récupération des centres et des events les suivant à supprimer
centres_events_a_suppr = events_centres[events_centres.centre_id.isin(centre_id_dupl)]

# Suppression de ces lignes, en vérifiant que les centres à supprimer n'ont pas amené à un but, sinon on les garde
events_centres.drop(centres_events_a_suppr[centres_events_a_suppr.But == "Non"].index, inplace = True)

In [22]:
# Aperçu des données
ex_but = centre_id_but.iloc[0]
events_centres[events_centres.centre_id == ex_but]

,team_centre,Centreur,centre_id,match_id_SB,period,index_event,x_loc,y_loc,event_id,type,...,pass_cross,x_shot,y_shot,z_shot,x_pass,y_pass,pass_body_part,player_id_SB,Tireur,But
12,156,15782.0,3,3894037,1,371,105.6,72.1,6bf75176-f3ab-4e0a-8110-a5790be58b16,Pass,...,1.0,NaN,NaN,NaN,107.9,34.4,Right Foot,15782.0,,Oui
13,156,15782.0,3,3894037,1,372,107.9,34.4,68b40e13-6877-4ca3-a782-f5c59262638f,Ball Receipt*,...,NaN,NaN,NaN,NaN,NaN,NaN,None,31308.0,,Non
14,156,15782.0,3,3894037,1,373,11.2,45.7,bd05c8c8-983f-457b-a6cd-edfa7fd7a26a,Duel,...,NaN,NaN,NaN,NaN,NaN,NaN,None,7252.0,,Non
15,156,15782.0,3,3894037,1,374,108.9,34.4,699b8341-8aa3-44b3-b839-105809acf579,Pass,...,NaN,NaN,NaN,NaN,107.8,48.3,None,31308.0,,Non
16,156,15782.0,3,3894037,1,375,107.8,48.3,0fdcd10f-6431-4246-9bc4-97e29694e249,Ball Receipt*,...,NaN,NaN,NaN,NaN,NaN,NaN,None,23527.0,,Non
17,156,15782.0,3,3894037,1,376,107.8,48.3,7244a2bb-35dd-4852-a103-1c6fea8d6533,Shot,...,NaN,120.0,43.3,0.8,NaN,NaN,None,23527.0,23527.0,Oui


---
Nous avons terminé le traitement des centres et des events suivant ces centres.  
Pour terminer ce programme, nous allons finaliser le dataframe "events_centres" en ajoutant des informations sur les matches liées à ces events telles que la date du match, la journée, la saison etc.  
Pour ce faire, nous allons importer les données "SB_matches" de Stats Bomb.

## 8) Ajout des informations sur les matches

In [23]:
# Importation des données "SB_matches"
colonnes = ["match_date", "match_week", "home_team_id_SB", "away_team_id_SB", "season", "competition", "match_id_SB"]
colonnes = ", ".join(colonnes)
req = cursor.execute(f"SELECT {colonnes} FROM SB_matches")
res = req.fetchall()
desc = req.description
SB_matches = pd.DataFrame(res)
SB_matches.columns = [i[0] for i in desc]

In [24]:
# Jointure des infos sur les matches avec le dataframe final
events_centres = pd.merge(events_centres, SB_matches, on = "match_id_SB")

In [25]:
# Aperçu des données
events_centres.head(3)

,team_centre,Centreur,centre_id,match_id_SB,period,index_event,x_loc,y_loc,event_id,type,...,pass_body_part,player_id_SB,Tireur,But,match_date,match_week,home_team_id_SB,away_team_id_SB,season,competition
0,147,26532.0,1,3894037,1,153,101.8,3.0,872df8f3-a750-4b61-8890-61a35e96b4a4,Pass,...,Left Foot,26532.0,,Non,2023-08-12,1,147,156,2023/2024,France - Ligue 1
1,147,26532.0,1,3894037,1,154,108.7,31.8,0f842e18-1363-4af3-8faa-6c9cce55b118,Ball Receipt*,...,None,63413.0,,Non,2023-08-12,1,147,156,2023/2024,France - Ligue 1
2,147,26532.0,1,3894037,1,155,3.7,42.5,a8a08a44-9486-4ce7-ab31-d6bd6f5cff23,Ball Recovery,...,None,24987.0,,Non,2023-08-12,1,147,156,2023/2024,France - Ligue 1


Et enfin, nous allons supprimer et renommer des colonnes afin de rendre ce dataframe plus cohérent et interprétable

In [26]:
# Liste contenant les colonnes à supprimer
colonnes_drop = ["shot_outcome", "team_id_SB", "match_id_SB", "player_id_SB", "index_event", "type", "pass_type", "period"]

# Suppression des colonnes
events_centres.drop(colonnes_drop, axis = 1, inplace = True)

# Dictionnaire contenant les colonnes à renommer en tant que clé, et les noms après renommage en tant que valeur
colonnes_rename = {"team_centre" : "Équipe", "pass_cross" : "Centre", "pass_body_part" : "Partie du corps", "match_date" : "Date",
                   "match_week" : "Journée", "minute" : "Minute", "season" : "Saison", "competition" : "Compétition",
                   "home_team_id_SB" : "Domicile", "away_team_id_SB" : "Extérieur"}

# Renommage des colonnes
events_centres.rename(colonnes_rename, axis = 1, inplace = True)

In [27]:
# Aperçu des données 1/2
colonnes_df = events_centres.columns.tolist()
events_centres[colonnes_df[:int(len(colonnes_df)/2)]].head(3)

,Équipe,Centreur,centre_id,x_loc,y_loc,event_id,Minute,Centre,x_shot,y_shot,z_shot
0,147,26532.0,1,101.8,3.0,872df8f3-a750-4b61-8890-61a35e96b4a4,3,1.0,NaN,NaN,NaN
1,147,26532.0,1,108.7,31.8,0f842e18-1363-4af3-8faa-6c9cce55b118,3,NaN,NaN,NaN,NaN
2,147,26532.0,1,3.7,42.5,a8a08a44-9486-4ce7-ab31-d6bd6f5cff23,3,NaN,NaN,NaN,NaN


In [28]:
# Aperçu des données 2/2
events_centres[colonnes_df[int(len(colonnes_df)/2):]].head(3)

,x_pass,y_pass,Partie du corps,Tireur,But,Date,Journée,Domicile,Extérieur,Saison,Compétition
0,116.4,37.6,Left Foot,,Non,2023-08-12,1,147,156,2023/2024,France - Ligue 1
1,NaN,NaN,None,,Non,2023-08-12,1,147,156,2023/2024,France - Ligue 1
2,NaN,NaN,None,,Non,2023-08-12,1,147,156,2023/2024,France - Ligue 1


---
Il ne nous reste plus qu'à écrire ces données dans une table de la base de données "database".  
Pour rappel, cette BDD contient les données que nous avons traitées, et que nous utilisons pour faire fonctionner l'application Streamlit

## 9) Écriture des données dans la base de données "database"

In [29]:
# Fermeture de la base de données "raw-database"
connect.close()

In [30]:
# Connexion à la base de données "database"
connect = sqlite3.connect("../../../Application_streamlit/database.db")

In [31]:
# Écriture des données du dataframe "events_centres" dans la BDD "database"
events_centres.to_sql(name = "zones_de_centre", con = connect, if_exists = "replace")

40640

In [32]:
# Fermeture de la BDD "database"
connect.close()